In [2]:
!pip install --user xlrd

In [2]:
# Setup constants if any




In [3]:
import pandas as pd
from io import BytesIO
import requests
import json
import xlrd 

from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime
from dateutil.parser import parse

from ingest.Connectors import Connectors

In [4]:
# The code was removed by DSX for sharing.

In [5]:
# The code was removed by DSX for sharing.

In [47]:

balSummaryDF = pd.read_csv(getFileFromObjectStorage('MizuhoPOC', 'FCM_MS_Balance_Summary___MAI_MyReport_All_Accounts_20151119114203.20170731.csv'))

# Drop rows & columns with all 'NaN' values, axis 0 is for row
# balEquityByCurrDF = balEquityByCurrDF.fillna('')

balSummaryDF.head(20)



balSummaryRenamedDF = balSummaryDF.rename(index=str, 
                                                columns={"0": "ACCT_NUM_1", 
                                                         "SecuredAccount": "ACCT_NUM",                                                         
                                                         "AccountName": "FUND_NAME",
                                                         "StatementDate": "AS_OF_DATE",
                                                         "Currency": "CURRENCY",
                                                         "SegregatedInd": "SEG_IND",
                                                         "CombinedInd": "COMB_IND",                                                          
                                                         "ConvertedInd": "CONV_IND", 
                                                         "FxRate": "FX_RATE",
                                                         "FxMultiplierInd": "FX_MULTIPLIER_IND",
                                                         "FXBaseCurrency": "FX_BASE_CURR",
                                                         "OpeningBalance": "OPEN_BAL",
                                                         "CashActivity":"CASH_ACTVY",                                                         
                                                         "FuturesCommission": "FUT_COMM",
                                                         "FuturesFees": "FUT_FEES",
                                                         "GrossPL":"GROSS_PL",
                                                         "NetPL": "NET_PL",                                                         
                                                         "ClosingBalance": "CLOSE_BAL",
                                                         "OpenTradeEquity": "OPEN_TRADE_EQUITY",
                                                         "PendingPS": "PENDING_PS",
                                                         "TotalEquity": "TOTAL_EQUITY",
                                                         "LMEDiscountMargin":"LME_DSCNT_MARGIN",
                                                         "InitialMargin":"INIT_MARGIN",
                                                         "NonCashCollateral":"NON_CASH_COLLATERAL",
                                                         "EquitySurplusDeficit":"EQUITY_SUR_DEFICIT",
                                                         "PendingPremium":"PEND_PREMIUM",
                                                         "ForecastEquity":"FORECAST_EQUITY",
                                                         "NetLiquidatingValue": "NET_LIQ_VAL",
                                                         "MTDNetRealizedOptionsProceeds":"MTD_NET_REALZ_OPT_PROCEEDS",
                                                         "MTDNetRealizedFuturePL":"MTD_NET_REALZ_FUT_PL",
                                                         "ProjectedSettlementAmt":"PROJ_SETTLE_AMT",
                                                         "ProjectedValueDate":"PROJ_VAL_DATE",
                                                         "PendingNonCashAmt":"PEND_NONCASH_AMT",
                                                         "CashNonCashAmt":"CASH_NONCASH_AMT",
                                                         "CollateralSurplusDeficit":"COLLATERAL_SUR_DEFICIT",
                                                         "CashCollateralAmt":"CASH_COLLATERAL_AMT",
                                                         "RealizedVM":"REALIZE_VM",
                                                         "VMAdjustments":"VM_ADJUST",
                                                         "TotalNetPL":"TOT_NET_PL",
                                                         "PendingCash":"PEND_CASH",
                                                         "PendingCashValueDate":"PEND_CASH_VAL_DATE",
                                                         "ExcessMarginPercentage":"EXCESS_MARGIN_PCT",
                                                         "MTDMTMAmt":"MTD_MTM_AMT"
                                                        })

# Convert the datetimeindex object to date

# Format the date string yyyymmdd to a date yyyy-mm-dd
balSummaryRenamedDF['AS_OF_DATE'] = balSummaryRenamedDF['AS_OF_DATE'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
balSummaryRenamedDF['AS_OF_DATE'] = pd.DatetimeIndex(balSummaryRenamedDF['AS_OF_DATE']).date

balSummaryRenamedDF[['FX_RATE', 'OPEN_BAL','CASH_ACTVY','FUT_COMM', 'FUT_FEES', 'GROSS_PL', 'NET_PL', 'CLOSE_BAL', 'OPEN_TRADE_EQUITY', 'PENDING_PS', 'TOTAL_EQUITY', 'LME_DSCNT_MARGIN', 'INIT_MARGIN', 'NON_CASH_COLLATERAL', 'EQUITY_SUR_DEFICIT', 'PEND_PREMIUM', 'FORECAST_EQUITY','NET_LIQ_VAL', 'MTD_NET_REALZ_OPT_PROCEEDS', 'MTD_NET_REALZ_FUT_PL', 'PROJ_SETTLE_AMT', 'PEND_NONCASH_AMT', 'CASH_NONCASH_AMT', 'COLLATERAL_SUR_DEFICIT', 'CASH_COLLATERAL_AMT', 'REALIZE_VM', 'VM_ADJUST', 'TOT_NET_PL', 'PEND_CASH', 'EXCESS_MARGIN_PCT', 'MTD_MTM_AMT']] = \
balSummaryRenamedDF[['FX_RATE', 'OPEN_BAL','CASH_ACTVY','FUT_COMM', 'FUT_FEES', 'GROSS_PL', 'NET_PL', 'CLOSE_BAL', 'OPEN_TRADE_EQUITY', 'PENDING_PS', 'TOTAL_EQUITY', 'LME_DSCNT_MARGIN', 'INIT_MARGIN', 'NON_CASH_COLLATERAL', 'EQUITY_SUR_DEFICIT', 'PEND_PREMIUM', 'FORECAST_EQUITY','NET_LIQ_VAL', 'MTD_NET_REALZ_OPT_PROCEEDS', 'MTD_NET_REALZ_FUT_PL', 'PROJ_SETTLE_AMT', 'PEND_NONCASH_AMT', 'CASH_NONCASH_AMT', 'COLLATERAL_SUR_DEFICIT', 'CASH_COLLATERAL_AMT', 'REALIZE_VM', 'VM_ADJUST', 'TOT_NET_PL', 'PEND_CASH', 'EXCESS_MARGIN_PCT', 'MTD_MTM_AMT']].astype(float)

balSummaryRenamedDF.head(35)
# print balSummaryRenamedDF.dtypes

,ACCT_NUM_1,ACCT_NUM,FUND_NAME,AS_OF_DATE,CURRENCY,SEG_IND,COMB_IND,CONV_IND,FX_RATE,FX_MULTIPLIER_IND,...,CASH_NONCASH_AMT,COLLATERAL_SUR_DEFICIT,CASH_COLLATERAL_AMT,REALIZE_VM,VM_ADJUST,TOT_NET_PL,PEND_CASH,PEND_CASH_VAL_DATE,EXCESS_MARGIN_PCT,MTD_MTM_AMT
0,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,USD,Y,N,N,0.009071,N,...,0,0,0,0,0,0.00,0.00,NaN,0,799261.12
1,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,JPY,Y,N,N,1.000000,N,...,0,0,0,0,0,0.00,0.00,NaN,0,0.00
2,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,JPY,N,N,Y,1.000000,N,...,0,0,0,0,0,0.00,0.00,NaN,0,0.00
3,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,JPY,N,N,N,1.000000,N,...,0,0,0,0,0,0.00,0.00,NaN,0,0.00
4,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,USD,N,N,N,0.009071,N,...,0,0,0,0,0,0.00,0.00,NaN,0,0.00
5,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,JPY,Y,Y,Y,1.000000,N,...,0,0,0,0,0,0.00,0.00,NaN,0,88111687.00
6,052BAHDH4,05ABAHDH5,WILSHIRE INSTITUTIONAL MASTER FUND II SPC-US I...,2017-07-31,JPY,Y,N,Y,1.000000,N,...,0,0,0,0,0,0.00,0.00,NaN,0,88111687.00
7,052BAD2W2,05ABAD2W3,THE ADVISORS' INNER CIRCLE FUND III-FIERA CAPITAL,2017-07-31,USD,Y,N,N,1.000000,Y,...,0,0,0,0,0,27638.03,0.00,NaN,0,144005.96
8,052BAD2W2,05ABAD2W3,THE ADVISORS' INNER CIRCLE FUND III-FIERA CAPITAL,2017-07-31,CAD,N,N,N,1.245300,N,...,0,0,0,0,0,-10746.95,-2117.80,NaN,0,5742.50
9,052BAD2W2,05ABAD2W3,THE ADVISORS' INNER CIRCLE FUND III-FIERA CAPITAL,2017-07-31,AUD,Y,N,N,0.799750,Y,...,0,0,0,0,0,0.00,0.00,NaN,0,0.00


In [39]:
spark = SparkSession.builder.getOrCreate()  

def build_schema():
    """Build and return a schema to use for the sample data."""
    schema = StructType(
        [
            StructField("ACCT_NUM_1", StringType(), True),
            StructField("ACCT_NUM", StringType(), False),
            StructField("FUND_NAME", StringType(), False),  
            StructField("AS_OF_DATE", DateType(), False),
            StructField("CURRENCY", StringType(), True),
            StructField("SEG_IND", StringType(), False),
            StructField("COMB_IND", StringType(), True),
            StructField("CONV_IND", StringType(), True),
            StructField("FX_RATE", DoubleType(), True),   
            StructField("FX_MULTIPLIER_IND", StringType(), True),
            StructField("FX_BASE_CURR", StringType(), True),            
            StructField("OPEN_BAL", DoubleType(), True),   
            StructField("CASH_ACTVY", DoubleType(), True),   
            StructField("FUT_COMM", DoubleType(), True),   
            StructField("FUT_FEES", DoubleType(), True),               
            StructField("GROSS_PL", DoubleType(), True),   
            StructField("NET_PL", DoubleType(), True),               
            StructField("CLOSE_BAL", DoubleType(), True),               
            StructField("OPEN_TRADE_EQUITY", DoubleType(), True),   
            StructField("PENDING_PS", DoubleType(), True),   
            StructField("TOTAL_EQUITY", DoubleType(), True),               
            StructField("LME_DSCNT_MARGIN", DoubleType(), True),               
            StructField("INIT_MARGIN", DoubleType(), True),     
            StructField("NON_CASH_COLLATERAL", DoubleType(), True), 
            StructField("EQUITY_SUR_DEFICIT", DoubleType(), True), 
            StructField("PEND_PREMIUM", DoubleType(), True),             
            StructField("FORECAST_EQUITY", DoubleType(), True),             
            StructField("NET_LIQ_VAL", DoubleType(), True),             
            StructField("MTD_NET_REALZ_OPT_PROCEEDS", DoubleType(), True),             
            StructField("MTD_NET_REALZ_FUT_PL", DoubleType(), True),             
            StructField("PROJ_SETTLE_AMT", DoubleType(), True),                 
            StructField("PROJ_VAL_DATE", StringType(), True),
            StructField("PEND_NONCASH_AMT", DoubleType(), True),             
            StructField("CASH_NONCASH_AMT", DoubleType(), True),             
            StructField("COLLATERAL_SUR_DEFICIT", DoubleType(), True),             
            StructField("CASH_COLLATERAL_AMT", DoubleType(), True),             
            StructField("REALIZE_VM", DoubleType(), True),             
            StructField("VM_ADJUST", DoubleType(), True),             
            StructField("TOT_NET_PL", DoubleType(), True),             
            StructField("PEND_CASH", DoubleType(), True),             
            StructField("PEND_CASH_VAL_DATE", StringType(), True),
            StructField("EXCESS_MARGIN_PCT", DoubleType(), True), 
            StructField("MTD_MTM_AMT", DoubleType(), True)
        ]
    )
    return schema


balSummarySparkDF = spark.createDataFrame(balSummaryRenamedDF, schema=build_schema())

# balSummarySparkDF.printSchema()

balSummarySparkDF.show(1)




root
 |-- ACCT_NUM_1: string (nullable = true)
 |-- ACCT_NUM: string (nullable = false)
 |-- FUND_NAME: string (nullable = false)
 |-- AS_OF_DATE: date (nullable = false)
 |-- CURRENCY: string (nullable = true)
 |-- SEG_IND: string (nullable = false)
 |-- COMB_IND: string (nullable = true)
 |-- CONV_IND: string (nullable = true)
 |-- FX_RATE: double (nullable = true)
 |-- FX_MULTIPLIER_IND: string (nullable = true)
 |-- FX_BASE_CURR: string (nullable = true)
 |-- OPEN_BAL: double (nullable = true)
 |-- CASH_ACTVY: double (nullable = true)
 |-- FUT_COMM: double (nullable = true)
 |-- FUT_FEES: double (nullable = true)
 |-- GROSS_PL: double (nullable = true)
 |-- NET_PL: double (nullable = true)
 |-- CLOSE_BAL: double (nullable = true)
 |-- OPEN_TRADE_EQUITY: double (nullable = true)
 |-- PENDING_PS: double (nullable = true)
 |-- TOTAL_EQUITY: double (nullable = true)
 |-- LME_DSCNT_MARGIN: double (nullable = true)
 |-- INIT_MARGIN: double (nullable = true)
 |-- NON_CASH_COLLATERAL: doub

In [40]:
dashDBloadOptions = { 
                    Connectors.DASHDB.HOST              : dashCredentials["host"],
                    Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                    Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                    Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                    Connectors.DASHDB.SOURCE_TABLE_NAME : dashCredentials["REF_FUND_MAPPING_TABLE"],
}

refFundMappingDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
refFundMappingDF.printSchema()
refFundMappingDF.show(1)

root
 |-- ALADDIN_ID: string (nullable = true)
 |-- FUND_NAME: string (nullable = true)
 |-- FUTURES_ACCT_ID: string (nullable = true)
 |-- BTIG_ID: string (nullable = true)
 |-- FUND_ID: string (nullable = true)
 |-- NEWEDGE_ID: string (nullable = true)
 |-- BASE: string (nullable = true)
 |-- BARC_ID: string (nullable = true)

+----------+------------------+---------------+-------+---------+----------+----+-------+
|ALADDIN_ID|         FUND_NAME|FUTURES_ACCT_ID|BTIG_ID|  FUND_ID|NEWEDGE_ID|BASE|BARC_ID|
+----------+------------------+---------------+-------+---------+----------+----+-------+
|     I-CJF|CRYSTAL JAPAN FUND|          C6500|       |I-ASIAPAC|     72590|  JY|  4902C|
+----------+------------------+---------------+-------+---------+----------+----+-------+
only showing top 1 row



In [53]:
# balSummarySparkDF.show(35)

balSummaryJoinSparkDF = balSummarySparkDF.join(refFundMappingDF, 
                                               trim(balSummarySparkDF.FUND_NAME) == refFundMappingDF.FUND_NAME, "inner")\
                                        .select(
                                            refFundMappingDF.ALADDIN_ID.alias("FUND_ID"),
                                            balSummarySparkDF.ACCT_NUM_1,
                                            balSummarySparkDF.ACCT_NUM,
                                            balSummarySparkDF.FUND_NAME,
                                            balSummarySparkDF.AS_OF_DATE,
                                            balSummarySparkDF.CURRENCY,
                                            balSummarySparkDF.SEG_IND,
                                            balSummarySparkDF.COMB_IND,
                                            balSummarySparkDF.CONV_IND,
                                            balSummarySparkDF.FX_RATE,
                                            balSummarySparkDF.FX_MULTIPLIER_IND,
                                            balSummarySparkDF.FX_BASE_CURR,
                                            balSummarySparkDF.OPEN_BAL,
                                            balSummarySparkDF.CASH_ACTVY,
                                            balSummarySparkDF.FUT_COMM,
                                            balSummarySparkDF.FUT_FEES,
                                            balSummarySparkDF.GROSS_PL,
                                            balSummarySparkDF.NET_PL,
                                            balSummarySparkDF.CLOSE_BAL,
                                            balSummarySparkDF.OPEN_TRADE_EQUITY,
                                            balSummarySparkDF.PENDING_PS,
                                            balSummarySparkDF.TOTAL_EQUITY,
                                            balSummarySparkDF.LME_DSCNT_MARGIN,
                                            balSummarySparkDF.INIT_MARGIN,
                                            balSummarySparkDF.NON_CASH_COLLATERAL,
                                            balSummarySparkDF.EQUITY_SUR_DEFICIT,
                                            balSummarySparkDF.PEND_PREMIUM,
                                            balSummarySparkDF.FORECAST_EQUITY,
                                            balSummarySparkDF.NET_LIQ_VAL,
                                            balSummarySparkDF.MTD_NET_REALZ_OPT_PROCEEDS,
                                            balSummarySparkDF.MTD_NET_REALZ_FUT_PL,
                                            balSummarySparkDF.PROJ_SETTLE_AMT,
                                            balSummarySparkDF.PROJ_VAL_DATE,
                                            balSummarySparkDF.PEND_NONCASH_AMT,
                                            balSummarySparkDF.CASH_NONCASH_AMT,
                                            balSummarySparkDF.COLLATERAL_SUR_DEFICIT,
                                            balSummarySparkDF.CASH_COLLATERAL_AMT,
                                            balSummarySparkDF.REALIZE_VM,
                                            balSummarySparkDF.VM_ADJUST,
                                            balSummarySparkDF.TOT_NET_PL,
                                            balSummarySparkDF.PEND_CASH,
                                            balSummarySparkDF.PEND_CASH_VAL_DATE,
                                            balSummarySparkDF.EXCESS_MARGIN_PCT,
                                            balSummarySparkDF.MTD_MTM_AMT
                                            
                                            )

# balSummaryJoinSparkDF.show(10)



+-------+----------+---------+-------------------+----------+--------+-------+--------+--------+--------+-----------------+------------+------------+----------+---------+--------+---------+---------+------------+-----------------+----------+------------+----------------+-------------+-------------------+------------------+------------+---------------+------------+--------------------------+--------------------+---------------+-------------+----------------+----------------+----------------------+-------------------+----------+---------+----------+-----------+------------------+-----------------+------------+
|FUND_ID|ACCT_NUM_1| ACCT_NUM|          FUND_NAME|AS_OF_DATE|CURRENCY|SEG_IND|COMB_IND|CONV_IND| FX_RATE|FX_MULTIPLIER_IND|FX_BASE_CURR|    OPEN_BAL|CASH_ACTVY| FUT_COMM|FUT_FEES| GROSS_PL|   NET_PL|   CLOSE_BAL|OPEN_TRADE_EQUITY|PENDING_PS|TOTAL_EQUITY|LME_DSCNT_MARGIN|  INIT_MARGIN|NON_CASH_COLLATERAL|EQUITY_SUR_DEFICIT|PEND_PREMIUM|FORECAST_EQUITY| NET_LIQ_VAL|MTD_NET_REALZ_OPT_

In [54]:
# Connection to Dash DB for writing the data
dashdbsaveoption = {
                     Connectors.DASHDB.HOST              : dashCredentials["host"],
                     Connectors.DASHDB.DATABASE          : dashCredentials["db"],
                     Connectors.DASHDB.USERNAME          : dashCredentials["username"],
                     Connectors.DASHDB.PASSWORD          : dashCredentials["password"],
                     Connectors.DASHDB.TARGET_TABLE_NAME : dashCredentials["tableName"],
                     Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' 
}

balSummaryJoinSparkDF.printSchema()

saveDashDBDF = balSummaryJoinSparkDF.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

root
 |-- FUND_ID: string (nullable = true)
 |-- ACCT_NUM_1: string (nullable = true)
 |-- ACCT_NUM: string (nullable = false)
 |-- FUND_NAME: string (nullable = false)
 |-- AS_OF_DATE: date (nullable = false)
 |-- CURRENCY: string (nullable = true)
 |-- SEG_IND: string (nullable = false)
 |-- COMB_IND: string (nullable = true)
 |-- CONV_IND: string (nullable = true)
 |-- FX_RATE: double (nullable = true)
 |-- FX_MULTIPLIER_IND: string (nullable = true)
 |-- FX_BASE_CURR: string (nullable = true)
 |-- OPEN_BAL: double (nullable = true)
 |-- CASH_ACTVY: double (nullable = true)
 |-- FUT_COMM: double (nullable = true)
 |-- FUT_FEES: double (nullable = true)
 |-- GROSS_PL: double (nullable = true)
 |-- NET_PL: double (nullable = true)
 |-- CLOSE_BAL: double (nullable = true)
 |-- OPEN_TRADE_EQUITY: double (nullable = true)
 |-- PENDING_PS: double (nullable = true)
 |-- TOTAL_EQUITY: double (nullable = true)
 |-- LME_DSCNT_MARGIN: double (nullable = true)
 |-- INIT_MARGIN: double (nullable